### **Notebook I: Penetration Paper Calculations** Notebook for the "[Full Paper Title]" manuscript.

* **Author:** Amir Babaei: amir.babaei@ieee.org, Frankfurt University of Applied Sciences
* **Date:** July 2025

**Objective:** This notebook contains the primary loop detectors data analysis and filtering
1. Data loading, integeration, and pre-processing
2. Anomaly detection and filtering
3. Generating output needed for further processing in the pipeline

#### Data loading, integeration, and pre-processing
##### In this section we:
* load the loop detector data for the months: 11.24, 12.24, 1.25, 2.25 
* for both 1.Flow and 2. Occupancy: 
    * create a standardized pivot format for further processing: [timestamps] X [locations] 

In [1]:
# Load the variables from .env file

import pandas as pd
import glob
import os
from dotenv import load_dotenv
load_dotenv()

loop_data_location = os.getenv("loop_data_location")


In [2]:
# Load the data for 11.24

df_nov = pd.read_csv(loop_data_location + 'november_loop_data.csv')

df_nov.head()

,id,measurement_site_reference_id,measurement_time,vehicle_flow_rate,occupancy_percentage,created_at
0,1,18371006[D30b],2024-10-31 12:28:00+00,240.0,6.82,2024-10-31 12:29:11.391584
1,2,18845003[D3a],2024-10-31 12:28:00+00,120.0,3.12,2024-10-31 12:29:11.39426
2,3,18678002[D2 (KFZ1)],2024-10-31 12:28:00+00,180.0,21.02,2024-10-31 12:29:11.395204
3,4,18371007[D40a],2024-10-31 12:28:00+00,180.0,32.83,2024-10-31 12:29:11.396018
4,5,18372008[D8 (5)],2024-10-31 12:28:00+00,60.0,23.18,2024-10-31 12:29:11.396838


In [3]:
# check the nans in the data
print("### NaN Values in November Data ###")
print(df_nov.isna().sum())  

# Check the distributions of the nans in various columns
print("### NaN Distribution in November Data ###")
print(df_nov.isna().mean().sort_values(ascending=False))


### NaN Values in November Data ###
id                                    0
measurement_site_reference_id         0
measurement_time                      0
vehicle_flow_rate                327548
occupancy_percentage             355737
created_at                            0
dtype: int64
### NaN Distribution in November Data ###
occupancy_percentage             0.003618
vehicle_flow_rate                0.003331
measurement_site_reference_id    0.000000
id                               0.000000
measurement_time                 0.000000
created_at                       0.000000
dtype: float64


In [4]:
# Load the data for 12.24, 1.25, and 2.25


# Define the base directory for your data
BASE_PATH = loop_data_location

# Define the month patterns you want to load
MONTH_PATTERNS = [
    'traffic_202412*.csv', # December 2024
    'traffic_202501*.csv', # January 2025
    'traffic_202502*.csv'  # February 2025
]

# Use glob to find all file paths that match the patterns
all_files = []
for pattern in MONTH_PATTERNS:
    all_files.extend(glob.glob(os.path.join(BASE_PATH, pattern)))

# Sort the file list to ensure chronological order
all_files.sort()

# Load and concatenate all the found CSVs into a single DataFrame
df_combined = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)


print(f"Found and loaded {len(all_files)} data files.")
print(f"Shape of the combined DataFrame: {df_combined.shape}")

print("\nCombined DataFrame `df_combined` is ready for analysis.")

# Display the first few rows of the combined DataFrame
df_combined.head()


Found and loaded 88 data files.
Shape of the combined DataFrame: (320429900, 4)

Combined DataFrame `df_combined` is ready for analysis.


,measurementSiteReferenceId,measurementTimeDefault,vehicleFlowRate,occupancyPercentage
0,18371006[D30b],2024-12-03T13:41:00.000+01:00,180.0,5.783333
1,18845003[D3a],2024-12-03T13:41:00.000+01:00,0.0,0.000000
2,18678002[D2 (KFZ1)],2024-12-03T13:41:00.000+01:00,420.0,5.700000
3,18371007[D40a],2024-12-03T13:41:00.000+01:00,180.0,11.533334
4,18372008[D8 (5)],2024-12-03T13:41:00.000+01:00,180.0,4.866667


In [5]:
# check the nans in the data
print("### NaN Values in combined Data ###")
print(df_combined.isna().sum())

# Check the distributions of the nans in various columns
print("### NaN Distribution in combined Data ###")
print(df_combined.isna().mean().sort_values(ascending=False))

### NaN Values in combined Data ###
measurementSiteReferenceId       0
measurementTimeDefault           0
vehicleFlowRate               7581
occupancyPercentage              1
dtype: int64
### NaN Distribution in combined Data ###
vehicleFlowRate               2.365884e-05
occupancyPercentage           3.120807e-09
measurementTimeDefault        0.000000e+00
measurementSiteReferenceId    0.000000e+00
dtype: float64


In [6]:
## Combining DataFrames with Standardized Column Names

# 1. Define the mapping to standardize column names
column_rename_map = {
    'measurementSiteReferenceId': 'measurement_site_reference_id',
    'measurementTimeDefault':     'measurement_time',
    'vehicleFlowRate':            'vehicle_flow_rate',
    'occupancyPercentage':        'occupancy_percentage'
}

# 2. Rename the columns of your existing df_combined DataFrame
df_combined_renamed = df_combined.rename(columns=column_rename_map)

# 3. Concatenate the renamed df_combined with your existing df_nov
df_final = pd.concat([df_combined_renamed, df_nov], ignore_index=True)

# 4. Standardize the measurement_time column using format='mixed'
# This correctly handles the multiple string formats in the column.
df_final['measurement_time'] = pd.to_datetime(
    df_final['measurement_time'], 
    format='mixed', 
    utc=True
)

# 5. Drop unnecessary columns
df_final.drop(columns=['created_at','id'], inplace=True)

# 6. Display the head and info of the final DataFrame to verify the merge
print("### Final Combined DataFrame Head ###")
print(df_final.head())
print("\n### Final Combined DataFrame Info ###")
df_final.info()

### Final Combined DataFrame Head ###
  measurement_site_reference_id          measurement_time  vehicle_flow_rate  \
0                18371006[D30b] 2024-12-03 12:41:00+00:00              180.0   
1                 18845003[D3a] 2024-12-03 12:41:00+00:00                0.0   
2           18678002[D2 (KFZ1)] 2024-12-03 12:41:00+00:00              420.0   
3                18371007[D40a] 2024-12-03 12:41:00+00:00              180.0   
4              18372008[D8 (5)] 2024-12-03 12:41:00+00:00              180.0   

   occupancy_percentage  
0              5.783333  
1              0.000000  
2              5.700000  
3             11.533334  
4              4.866667  

### Final Combined DataFrame Info ###
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418766770 entries, 0 to 418766769
Data columns (total 4 columns):
 #   Column                         Dtype              
---  ------                         -----              
 0   measurement_site_reference_id  object             
 1

In [ ]:
# Find and display rows where the combination of time and location is duplicated
duplicate_rows = df_final[df_final.duplicated(subset=['measurement_time', 'measurement_site_reference_id'], keep=False)]

print(f"Found {len(duplicate_rows)} rows that are part of a duplicate set.")
# Sort to see the duplicates next to each other
print(duplicate_rows.sort_values(by=['measurement_time', 'measurement_site_reference_id']).head(50))

##### Since we have a duplicate issue with NaNs, first we try to resolve the issue and then we create the pivot tables.

In [8]:
# --- 1. Consolidate Duplicate Rows ---
print("Handling systematic duplicates...")

# Sort by time, location, and then by vehicle_flow_rate.
# This pushes rows with NaN in 'vehicle_flow_rate' to the top of each duplicate group.
df_final.sort_values(
    by=['measurement_time', 'measurement_site_reference_id', 'vehicle_flow_rate'],
    na_position='first', # Explicitly place NaNs first
    inplace=True
)

# Drop duplicates based on time and location, keeping only the 'last' entry.
# The 'last' entry is the one with the complete vehicle_flow_rate data.
df_final.drop_duplicates(
    subset=['measurement_time', 'measurement_site_reference_id'],
    keep='last',
    inplace=True
)

print(f"DataFrame shape after consolidation: {df_final.shape}")


# --- 2. Create Pivoted DataFrames (Now using the faster 'pivot') ---
# Since the duplicates are resolved, you can now use the original, more efficient 'pivot'
print("\nPivoting data for vehicle flow rate...")
df_flow = df_final.pivot(
    index='measurement_time',
    columns='measurement_site_reference_id',
    values='vehicle_flow_rate'
)
print("Pivoting for flow complete.")

print("\nPivoting data for occupancy percentage...")
df_occ = df_final.pivot(
    index='measurement_time',
    columns='measurement_site_reference_id',
    values='occupancy_percentage'
)
print("Pivoting for occupancy complete.")


# --- 3. Display Results ---
print("\n\n### Vehicle Flow DataFrame (df_flow) ###")
print(df_flow.head())

Handling systematic duplicates...
DataFrame shape after consolidation: (417973525, 4)

Pivoting data for vehicle flow rate...
Pivoting for flow complete.

Pivoting data for occupancy percentage...
Pivoting for occupancy complete.


### Vehicle Flow DataFrame (df_flow) ###
measurement_site_reference_id  10002001[D10 (KFZ1)]  10002002[D20 (KFZ2)]  \
measurement_time                                                            
2024-10-31 12:28:00+00:00                      60.0                 180.0   
2024-10-31 12:29:00+00:00                      60.0                 240.0   
2024-10-31 12:30:00+00:00                     240.0                 180.0   
2024-10-31 12:32:00+00:00                     240.0                 540.0   
2024-10-31 12:33:00+00:00                     300.0                 300.0   

measurement_site_reference_id  10002003[D3a (KFZ3)]  10002004[D3 (KFZ3)]  \
measurement_time                                                           
2024-10-31 12:28:00+00:00          

In [11]:
# Save the pivoted DataFrames to CSV files
df_flow.to_csv('data/loop/df_flow_pivoted.csv')
df_occ.to_csv('data/loop/df_occ_pivoted.csv')